In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data with Start-Stop-Markers

This tutorial shows how to read and decode BBCI data with start and stop markers. The data loading part is more complicated and it is advised to read the other tutorials before.

## Setup logging to see outputs

In [ ]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()


## Load and preprocess data

This is a bit more complicated than before since we have to add breaks etc. Here I now opt to add breaks do all preprocessings per run and only later combine them together.

In [ ]:
import numpy as np
from braindecode.datautil.splitters import concatenate_sets
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne, add_breaks
from braindecode.datasets.bbci import load_bbci_sets_from_folder
from copy import deepcopy
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import lowpass_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

def create_cnts(folder, runs, name_to_start_code, name_to_stop_code, break_start_offset_ms,
              break_stop_offset_ms, break_start_code, break_stop_code):
    # Load data
    cnts = load_bbci_sets_from_folder(folder, runs)
    
    # Now do some preprocessings:
    # Adding breaks, resampling to 250 Hz, lowpass below 38, eponential standardization
    break_start_code = -1
    break_stop_code = -2
    
    new_cnts = []
    for cnt in cnts:
        # Only take some channels 
        #cnt = cnt.drop_channels(['STI 014']) # This would remove stimulus channel
        cnt = cnt.pick_channels(['C3', 'CPz' 'C4'])
        # add breaks
        new_events = add_breaks(
            np.array(cnt.info['events'])[:, [0,2]], cnt.info['sfreq'],
            break_start_code=break_start_code, break_stop_code=break_stop_code,
            name_to_start_codes=name_to_start_code, name_to_stop_codes=name_to_stop_code,
            min_break_length_ms=5000, max_break_length_ms=9000)
        n_break_start_offset = int(cnt.info['sfreq'] * break_start_offset_ms / 1000.0)
        n_break_stop_offset = int(cnt.info['sfreq'] * break_stop_offset_ms / 1000.0)
        # lets add some offset to break start and stop
        # new_events[:,2] contains event codes, new_events[:,0] the sample indices
        # new_events[:,1] is always 0 for my loading of BBCI data
        new_events[new_events[:,1] == break_start_code, 0] += n_break_start_offset
        # 0.5 sec for break stop
        new_events[new_events[:,1] == break_stop_code, 0] +=  n_break_stop_offset
        # add a middle column with zeros again
        new_events_for_mne = np.zeros((len(new_events), 3))
        new_events_for_mne[:,[0,2]] = new_events
        cnt.info['events'] = new_events_for_mne
        log.info("Preprocessing....")
        cnt = mne_apply(lambda a: lowpass_cnt(a, 38,cnt.info['sfreq'], axis=1), cnt)
        cnt = resample_cnt(cnt, 250)
        # mne apply will apply the function to the data (a 2d-numpy-array)
        # have to transpose data back and forth, since
        # exponential_running_standardize expects time x chans order
        # while mne object has chans x time order
        cnt = mne_apply(lambda a: exponential_running_standardize(
            a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
            cnt)
        new_cnts.append(cnt)
    return new_cnts

In [ ]:
from collections import OrderedDict
name_to_start_code = OrderedDict([('Right Hand', 1), ('Feet', 4),
            ('Rotation', 8), ('Words', [10])])

name_to_stop_code = OrderedDict([('Right Hand', [20,21,22,23,24,28,30]),
        ('Feet', [20,21,22,23,24,28,30]),
        ('Rotation', [20,21,22,23,24,28,30]), 
        ('Words', [20,21,22,23,24,28,30])])

break_start_offset_ms = 1000
break_stop_offset_ms = -500
# pick some numbers that were not used before/do not exist in markers
break_start_code = -1
break_stop_code = -2
train_runs = [1,2,3]
train_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/', 
                         train_runs,
                         name_to_start_code,
                         name_to_stop_code, break_start_offset_ms,
                         break_stop_offset_ms, break_start_code, break_stop_code)

name_to_code_with_breaks = deepcopy(name_to_start_code)
name_to_code_with_breaks['Break'] = break_start_code
name_to_stop_code_with_breaks = deepcopy(name_to_stop_code)
name_to_stop_code_with_breaks['Break'] = break_stop_code

In [ ]:
test_runs = [9,10]
test_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R09-10/', test_runs, name_to_start_code,
           name_to_stop_code, break_start_offset_ms,
              break_stop_offset_ms, break_start_code, break_stop_code)

## Create the model

We already create the model now, since we need to know the receptive field size for properly cutting out the data to predict. We need to cut out data starting at -receptive_field_size samples before the first sample we want to predict.

In [ ]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 650
in_chans = train_cnts[0].get_data().shape[0]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=5, input_time_length=input_time_length,
                        final_conv_length=29).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()
from braindecode.torch_ext.util import np_to_var
import numpy as np
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))
n_receptive_field = input_time_length - n_preds_per_input
receptive_field_ms = n_receptive_field * 1000.0 / train_cnts[0].info['sfreq']
print("Receptive field: {:d}/{:.2f} (samples/ms)".format(n_receptive_field,
                                                      receptive_field_ms))

### Create SignalAndTarget Sets

In [ ]:
train_sets = [create_signal_target_from_raw_mne(cnt, name_to_code_with_breaks, [-receptive_field_ms,0], 
                                         name_to_stop_code_with_breaks) for cnt in train_cnts]
train_set = concatenate_sets(train_sets)

In [ ]:
test_sets = [create_signal_target_from_raw_mne(cnt, name_to_code_with_breaks, [-receptive_field_ms,0], 
                                         name_to_stop_code_with_breaks) for cnt in test_cnts]
test_set = concatenate_sets(test_sets)

In [ ]:
train_set_2 = np.load('/data/schirrmr/schirrmr/BBCI_Data_Start_Stop-train-set-tmp-test.npy')

In [ ]:
import pickle
pickle.dump(train_set, open('/data/schirrmr/schirrmr/BBCI_Data_Start_Stop-train-set-tmp-test.npy', 'wb'))

In [ ]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Setup optimizer and iterator

In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters())

In [ ]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

## Setup Monitors, Loss function, Stop Criteria

In [ ]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression


loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2, keepdim=False), targets)

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [ ]:
exp.run()

We arrive only at 31% accuracy. With only 3 sensors and 3 training runs, cannot expect too much great performance :)